## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [31]:
# Place any addtional functions or constants you need here. 
from string import punctuation
from nltk.corpus import stopwords
import re
import emoji

# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

# Stopwords
sw = stopwords.words("english")

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

# It's handy to have a full set of emojis
all_language_emojis = set()

for country in emoji.UNICODE_EMOJI : 
    for em in emoji.UNICODE_EMOJI[country] : 
        all_language_emojis.add(em)

# and now our functions
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Return a list of 
    """

    # Fill in the correct values here.
    tokes = tokens.split()
    num_tokens = sum(map(len, (s.split() for s in tokes)))
    num_unique_tokens = len(set(w.lower() for w in tokes))
    lexical_diversity = num_unique_tokens / num_tokens
    num_characters = sum(list(map(len, tokes)))
    
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])
    
def is_emoji(s):
    return(s in all_language_emojis)

def contains_emoji(s):
    
    s = str(s)
    emojis = [ch for ch in s if is_emoji(ch)]

    return(len(emojis) > 0)


def remove_stop(tokens) :
    # modify this function to remove stopwords
    tokens_wo_sw = [word for word in tokens if not word in sw]
    return(tokens_wo_sw)
 
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    text = text.split()
    return(text)

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)


In [2]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [166]:
my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            Select text, party from conventions where speaker != 'Unknown'
                            ''')

for row in query_results :
    # store the results in convention_data
    text = str(prepare(row[0], my_pipeline))
    party = row[1]
    convention_data.append([text, party])

In [167]:
convention_data

[["['i’m', 'calling', 'full', 'session', '48th', 'quadrennial', 'national', 'convention', 'democratic', 'party', 'order', 'welcome', 'final', 'session', 'historic', 'memorable', 'convention', 'we’ve', 'called', '48th', 'quadrennial', 'democratic', 'national', 'convention', 'order']",
  'Democratic'],
 ["['every', 'four', 'years', 'come', 'together', 'reaffirm', 'democracy', 'year', 'we’ve', 'come', 'save']",
  'Democratic'],
 ["['fight', 'perfect', 'union', 'fighting', 'soul', 'country', 'lives', 'right', 'fight', 'real']",
  'Democratic'],
 ["['must', 'come', 'together', 'defeat', 'donald', 'trump', 'elect', 'joe', 'biden', 'kamala', 'harris', 'next', 'president', 'vice', 'president']",
  'Democratic'],
 ["['donald', 'trump', 'wrong', 'president', 'country', 'enough', 'time', 'prove', 'job', 'clearly', 'head', 'simply', 'cannot', 'need', 'us']",
  'Democratic'],
 ["['dad', 'healthy', '65', 'year', 'old', 'preexisting', 'condition', 'trusting', 'donald', 'trump', 'paid', 'life']",
  'D

Let's look at some random entries and see if they look right. 

In [168]:
random.choices(convention_data,k=10)

[["['didn’t', 'speaker', 'would', 'bring', 'gun', 'bill', 'floor', 'didn’t', 'speaker', 'would', 'bring', 'dreamer’s', 'issue', 'floor']",
  'Democratic'],
 ["['racist', 'coward', '…', 'speaker', '92', '013749', 'call', '…', 'cops', '…', 'speaker', '93', '013749', 'going', 'kill', '…', 'speaker', '94', '013749', 'i’m', 'rape', '…', 'speaker', '95', '013758', 'mark', 'mccloskey', 'says', 'family', 'threatened', 'violence']",
  'Republican'],
 ["['they’re', 'learning', 'conquered', 'pandemic', 'stood', 'united', 'racial', 'justice', 'built', 'economy', 'back', 'better', 'they’re', 'learning', 'leadership', 'gave', 'generation', 'fighting', 'chance', 'they’re', 'learning', 'us', 'resolve', 'unity', 'showed', 'forces', 'hatred', 'division', 'work', 'next', '11', 'weeks', 'night', 'despite', 'distance', 'came', 'together', 'nominate', 'joe', 'biden', 'president', 'united', 'states', 'nomination', 'i’m', 'honored', 'second']",
  'Democratic'],
 ["['want', 'go', 'law', 'school', 'want', 'help

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [227]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2396 as features in the model.


In [226]:
prepare(feature_words, my_pipeline)

['cared',
 'credit',
 'mental',
 'protest',
 'shaped',
 'doubt',
 'assault',
 'we’re',
 'global',
 'conversation',
 'liberties',
 'devastating',
 'examples',
 'towards',
 'number',
 'see',
 'better',
 'rebuilt',
 'sought',
 'iraq',
 'kill',
 'killing',
 'never',
 'dr',
 'income',
 '2017',
 'fierce',
 'point',
 'shift',
 'type',
 'haven’t',
 'states',
 'sentence',
 'coffee',
 'different',
 'marine',
 'capital',
 'grandparents',
 'secure',
 '222',
 'believed',
 'independent',
 'benefit',
 'break',
 'listened',
 'champion',
 'division',
 'deal',
 'ensure',
 'south',
 'conference',
 'dream',
 'process',
 'proud',
 'reform',
 '94108',
 '2018',
 'learning',
 'mob',
 'taxes',
 'time',
 'protective',
 'fair',
 '…',
 'virus',
 'time',
 'endless',
 'seek',
 'site',
 'daughters',
 '26',
 'couldn’t',
 'overseas',
 'pray',
 '100',
 'starting',
 'cover',
 'groups',
 '4',
 'cases',
 'chooses',
 'think',
 'clean',
 'jill',
 'delivered',
 'michigan',
 'eliminating',
 'deals',
 'hill',
 'patriots',
 'pa

In [284]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here 
    ret_dict = dict()
    #clean_fw = prepare(fw, my_pipeline)
    
    for tex in text.split():
        if tex in fw:
            ret_dict[tex]=True
    
    return(ret_dict)

In [239]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [285]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [287]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [288]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.546


In [290]:
classifier.show_most_informative_features(25)

Most Informative Features
                'media', = True           Republ : Democr =     35.9 : 1.0
              'radical', = True           Republ : Democr =     35.9 : 1.0
                'votes', = True           Democr : Republ =     22.8 : 1.0
          'enforcement', = True           Republ : Democr =     18.4 : 1.0
                'trump'] = True           Republ : Democr =     18.2 : 1.0
                'biden'] = True           Democr : Republ =     17.3 : 1.0
                'crime', = True           Republ : Democr =     17.2 : 1.0
             'freedoms', = True           Republ : Democr =     16.2 : 1.0
              'destroy', = True           Republ : Democr =     15.1 : 1.0
               'earned', = True           Republ : Democr =     14.1 : 1.0
                'china', = True           Republ : Democr =     14.1 : 1.0
                'bless', = True           Republ : Democr =     13.4 : 1.0
               'defund', = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

The classifier yielded an accuracy of .546.  This means that our model can predict the party of a candidate based on the feature words extracted from their speech 55% of the time.



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [245]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [246]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [254]:
my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.
for row in results:
    party = row[1]
    tweet = str(prepare(row[2], my_pipeline))
    tweet_data.append([tweet, party])


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [255]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [307]:
features = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

for tweet, party in tweet_data_sample :
    estimated_party = "Democratic"#classifier.classify(features)
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: ['bearlier', 'today', 'spoke', 'house', 'floor', 'abt', 'protecting', 'health', 'care', 'women', 'praised', 'ppmarmonte', 'work', 'central', 'coast', 'httpstcowqgtrzt7vv']
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: ['bgo', 'tribe', '#rallytogether', 'httpstco0nxutfl9l5']
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: ['bapparently', 'trump', 'thinks', 'easy', 'students', 'overwhelmed', 'crushing', 'burden', 'debt', 'pay', 'student', 'loans', '#trumpbudget', 'httpstcockyqo5t0qh']
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: ['bwexe2x80x99re', 'grateful', 'first', 'responders', 'rescue', 'personnel', 'firefighters', 'police', 'volunteers', 'working', 'tirelessly', 'keep', 'people', 'safe', 'provide', 'muchneeded', 'help', 'putting', 'lives', 'linennhttpstcoezpv0vmiz3']
Actual party is Republican and our classifer says Democr

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [308]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = "Democratic"
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [309]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 0, 'Democratic': 4334}),
             'Democratic': defaultdict(int,
                         {'Republican': 0, 'Democratic': 5668})})

### Reflections

I could not get this to work.  I tried for hours but to no avail.